In [1]:
import os, glob
import json

import numpy as np
import pandas as pd


In [11]:
lookup_path = "/home/craig.car/repos/chiron/chironata/data/cts_lookup_table.json"
with open(lookup_path) as f:
    lookup = json.load(f)

lookup["urn:cts:greekLit:tlg0640.tlg001"]

['alciphron_1896']

In [10]:
for path in ["/scratch/craig.car/english_trans-dev/tlg0640.tlg001.ogl-eng1.xml"]:
    prefix = os.path.splitext(path)[0]
    print(prefix)

/scratch/craig.car/english_trans-dev/tlg0640.tlg001.ogl-eng1


In [39]:
text_ctsurn = []
num_transl = []
transl_langs = []
num_src_sents = []
text_top_1s = []
text_top_5s = []
text_top_10s = []
num_tgt_sents = []
num_nulls_all_transl = []

not_transl_keys = ["num_src_sents", "num_translations", "text_top1", "text_top5", "text_top10"]

eval_dicts_dir = "/home/craig.car/repos/chiron/chironata/eval_datafiles/"
for eval_json_path in glob.iglob(eval_dicts_dir+"*.json"):
    ctsurn = os.path.splitext(os.path.basename(eval_json_path))[0]
    with open(eval_json_path) as f:
        eval_json = json.load(f)
    text_ctsurn.append(ctsurn)
    # print(eval_json["apolloniusRhodius_1_1791"].keys())
    num_transl.append(eval_json["num_translations"])
    num_src_sents.append(eval_json["num_src_sents"])
    text_top_1s.append(eval_json["text_top1"])
    text_top_5s.append(eval_json["text_top5"])
    text_top_10s.append(eval_json["text_top10"])
    text_langs = set()
    num_transl_sents = 0
    num_nulls_all_tranls = 0
    for key in eval_json.keys():
        if key not in not_transl_keys:
            # print(key)
            text_langs.add(eval_json[key]["tgt_lang"])
            num_transl_sents += eval_json[key]["num_tgt_sents"]
            num_nulls_all_tranls += eval_json[key]["num_nulls"]
            # print(num_transl_sents)
    transl_langs.append(text_langs)
    num_tgt_sents.append(num_transl_sents)
    num_nulls_all_transl.append(num_nulls_all_tranls)


In [40]:
print(len(text_ctsurn))
print(len(num_transl))
print(len(transl_langs))
print(len(num_src_sents))
print(len(text_top_1s))
print(len(text_top_5s))
print(len(text_top_10s))
print(len(num_tgt_sents))
print(len(num_nulls_all_transl))

709
709
709
709
709
709
709
709
709


In [41]:
eval_df = pd.DataFrame(text_ctsurn)
eval_df.columns = ["ctsurn"]

In [42]:
eval_df["number_src_sents"] = num_src_sents
eval_df["number_translations"] = num_transl
eval_df["translation_languages"] = transl_langs
eval_df["num_tgt_sents_all_translations"] = num_tgt_sents
eval_df["num_src2null_all_translations"] = num_nulls_all_transl
eval_df["text_wide_recall@1"] = text_top_1s
eval_df["text_wide_recall@5"] = text_top_5s
eval_df["text_wide_recall@10"] = text_top_10s

In [43]:
eval_df.head()

,ctsurn,number_src_sents,number_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10
0,urn:cts:greekLit:tlg0001.tlg001,1738,3,"{en, fr, it}",18326,0,0.178941,0.296318,0.353471
1,urn:cts:greekLit:tlg0003.tlg001,4419,7,"{en, fr, de}",53054,547,0.130379,0.194194,0.224873
2,urn:cts:greekLit:tlg0004.tlg001,6440,1,{en},11907,3,0.435714,0.654193,0.736801
3,urn:cts:greekLit:tlg0005.tlg001,549,5,"{en, fr, de}",23922,1,0.234244,0.409107,0.499089
4,urn:cts:greekLit:tlg0005.tlg002,94,5,"{en, fr, de}",23922,0,0.123404,0.200000,0.261702


In [44]:
eval_df.sort_values("text_wide_recall@10", ascending=[False])

,ctsurn,number_src_sents,number_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10
661,urn:cts:latinLit:phi1212.phi003,392,1,{en},8847,0,0.757653,0.989796,1.043367
595,urn:cts:latinLit:phi0474.phi055,2011,1,{en},5692,0,0.776231,0.945301,0.987568
666,urn:cts:latinLit:phi1348.abo011,517,2,"{fr, de}",13468,0,0.709865,0.913926,0.984526
433,urn:cts:greekLit:tlg0074.tlg004,282,1,{fr},3216,0,0.549645,0.872340,0.971631
323,urn:cts:greekLit:tlg0034.tlg001,406,1,{de},655,0,0.655172,0.874384,0.963054
...,...,...,...,...,...,...,...,...,...
463,urn:cts:greekLit:tlg0096.tlg002,6062,1,{de},7368,1,0.000660,0.006434,0.012042
596,urn:cts:latinLit:phi0474.phi056,6913,2,{de},11366,1032,0.001374,0.004484,0.008969
582,urn:cts:latinLit:phi0474.phi035,4047,2,{de},9216,10,0.000741,0.005313,0.008772
434,urn:cts:greekLit:tlg0074.tlg008,9,1,{fr},9106,0,0.000000,0.000000,0.000000


In [32]:
eval_df["number_src_sents"].sum()

579711

In [33]:
eval_df["num_tgt_sents_all_translations"].sum()

16428809

In [45]:
eval_df.sort_values("num_src2null_all_translations")

,ctsurn,number_src_sents,number_translations,translation_languages,num_tgt_sents_all_translations,num_src2null_all_translations,text_wide_recall@1,text_wide_recall@5,text_wide_recall@10
0,urn:cts:greekLit:tlg0001.tlg001,1738,3,"{en, fr, it}",18326,0,0.178941,0.296318,0.353471
426,urn:cts:greekLit:tlg0062.tlg065,204,1,{de},4222,0,0.348039,0.544118,0.696078
428,urn:cts:greekLit:tlg0062.tlg067,213,4,"{en, fr, de}",29213,0,0.322770,0.468310,0.534038
429,urn:cts:greekLit:tlg0062.tlg068,425,4,"{en, fr, de}",29213,0,0.327059,0.450000,0.525882
430,urn:cts:greekLit:tlg0062.tlg069,507,3,{de},9872,0,0.097962,0.174227,0.226167
...,...,...,...,...,...,...,...,...,...
465,urn:cts:greekLit:tlg0525.tlg001,7163,7,"{de, en, fr, it}",33812,11040,0.056401,0.076783,0.088770
531,urn:cts:greekLit:tlg2000.tlg001,12496,3,"{en, de}",14361,17386,0.029583,0.049536,0.058552
255,urn:cts:greekLit:tlg0016.tlg001,6998,13,"{en, fr, it}",73657,19690,0.107009,0.152186,0.173137
637,urn:cts:latinLit:phi0914.phi001,30809,4,"{fr, de}",38508,70085,0.085965,0.102738,0.109489
